In [1]:
import numpy as np
from matplotlib import pyplot as plt
import math
from numpy.linalg import inv
import numba
from numba import jit

\begin{equation}
i\hbar \partial_t\psi(x,t)=\left(-\frac{\hbar^2}{2m} \vec{\nabla}^2+\frac{1}{2}m\omega^2x^2 \right)\psi(x,t)=\hat{H}\psi(x,t) \tag{1}
\end{equation}

take out $\frac{\omega \hbar}{2}$ from right hand side:

\begin{equation}
i\hbar \partial_t\psi(x,t)=\frac{\omega \hbar}{2}\left(-\frac{\hbar}{m\omega} \vec{\nabla}^2+\frac{m\omega}{\hbar}x^2 \right)\psi(x,t)=\hat{H}\psi(x,t)  \tag{2}
\end{equation}

We devide above equation by $\hbar$. Then, introducing  $\alpha^2=\frac{m \omega}{\hbar}$, we get

\begin{equation}
i\partial_t\psi(x,t)=\frac{\omega}{2}\left(-\frac{1}{\alpha^2} \frac{\partial^2}{\partial x^2} +\alpha^2x^2 \right)\psi(x,t)=\frac{1}{\hbar}\hat{H}\psi(x,t)  \tag{3}
\end{equation}

Multipling  with   $2/\omega$ and introducing new variables $\xi=\alpha x$, $\tau=\frac{\omega t}{2}$ we get:

\begin{equation}
i\partial_\tau\psi=\left(- \frac{\partial^2}{\partial \xi^2} +\xi^2 \right)\psi=\frac{2}{\hbar\omega} \hat{H}\psi=\hat{\tilde{H}}\psi   \tag{4}
\end{equation}
where $\hat{\tilde{H}}=\frac{2}{\hbar\omega} \hat{H}$. Hence $\beta=\frac{2}{\hbar\omega}$.



In [10]:
ksi_l=-10
ksi_r=10
dksi=0.1    # correspond to delta x
dtau=0.005   #correspond to delta t
xi = np.arange(ksi_l, ksi_r + dksi, dksi)  # range is from -10 to 10 with 0.1 step. 10 is alos included
Nx = len(xi)    #number of steps
Nt=201   #number of times

I = np.eye(Nt,dtype=complex)


In [11]:
psi_0=np.zeros((Nt,Nx))
H=np.zeros((Nt,Nx),dtype=complex)

In [16]:
for n in range(Nx):
    if n>0:
        H[n,n-1]=-1/dksi**2
    if n<Nx-1:
        H[n,n+1]=-1/dksi**2
    H[n,n]=2/dksi**2+dksi**2*n**2

print(H)

[[ 200.  +0.j -100.  +0.j    0.  +0.j ...    0.  +0.j    0.  +0.j
     0.  +0.j]
 [-100.  +0.j  200.01+0.j -100.  +0.j ...    0.  +0.j    0.  +0.j
     0.  +0.j]
 [   0.  +0.j -100.  +0.j  200.04+0.j ...    0.  +0.j    0.  +0.j
     0.  +0.j]
 ...
 [   0.  +0.j    0.  +0.j    0.  +0.j ...  592.04+0.j -100.  +0.j
     0.  +0.j]
 [   0.  +0.j    0.  +0.j    0.  +0.j ... -100.  +0.j  596.01+0.j
  -100.  +0.j]
 [   0.  +0.j    0.  +0.j    0.  +0.j ...    0.  +0.j -100.  +0.j
   600.  +0.j]]


In [15]:
A=I-1j*H*dtau/2
B=I+1j*H*dtau/2

S_H=np.linalg.inv(A).dot(B)
print(S_H)

[[ 5.78947378e-001+7.18562228e-001j  2.83645847e-001-2.47086031e-001j
  -2.32831971e-002-7.81756119e-002j ... -1.09943305e-143-7.53936353e-144j
  -1.87066556e-144+1.49187831e-146j -2.14698427e-145+1.45618749e-145j]
 [ 2.83645847e-001-2.47086031e-001j  5.55635817e-001+6.40411324e-001j
   2.66136053e-001-2.49484012e-001j ...  8.16879316e-144-5.90560490e-143j
  -3.80100624e-144-7.45282466e-144j -1.01187185e-144-5.67556211e-145j]
 [-2.32831971e-002-7.81756119e-002j  2.66136053e-001-2.49484012e-001j
   5.53517966e-001+6.43726852e-001j ...  2.63556930e-142-7.79034675e-143j
   2.40795716e-143-3.01253384e-143j  4.61078388e-145-5.32827532e-144j]
 ...
 [-1.09943305e-143-7.53936353e-144j  8.16879316e-144-5.90560490e-143j
   2.63556930e-142-7.79034675e-143j ... -3.30000073e-001+9.15922562e-001j
   1.50645992e-001+5.16279103e-002j  2.13536075e-002-5.63108664e-003j]
 [-1.87066556e-144+1.49187831e-146j -3.80100624e-144-7.45282466e-144j
   2.40795716e-143-3.01253384e-143j ...  1.50645992e-001+5.162791

In [125]:
def boundary_cond(psi):
    psi_0=np.copy(psi)
    psi_0[:,0]=0
    psi_0[:,-1]=0
    return psi_0

In [126]:
#@numba.jit("c16[:,:](c16[:,:])",nopython=True,nogil=True)
def Crank_Nic_Ver(psii):
  
    psi_new=boundary_cond(psii)
    for n in range(Nt):
       for m in range(Nx):
           H[n,m]=-1.0/dksi**2*(psi_new[n,m-1]+psi_new[n,m+1]-2.0*psi_new[n,m])+dksi**2*n**2*psi_new[n,m]
           psi_new[n+1,m]= np.linalg.inv(I+1j/2.0*dtau*(H[n,m])).dot(I-1j/2.0*dtau*H[n,m])*psi_new[n,m]             
    return psi_new

In [127]:
result=Crank_Nic_Ver(psi_0)

ValueError: setting an array element with a sequence.